# AFL Predictions
The file we will run to get each rounds predictions!

In [1]:
#packages
import polars as pl
import pickle
import numpy as np
import pandas as pd
#Get Update function
from ipynb.fs.defs.AFLupdate import updateData, getCurrentRoundNumbers
#Get upcoming fixture
from ipynb.fs.defs.AFLfixture import createFixture

pl.Config.set_fmt_str_lengths(150);
pl.Config.set_tbl_cols(40);
pl.Config.set_tbl_rows(50);


/home/erin/Documents/AFL tipping/afl-env/lib/python3.12/site-packages/rpy2/robjects/lib/dplyr.py:27: UserWarning: This was designed against dplyr versions starting with 1.0 but you have 1.1.4
  warnings.warn(


In [2]:
#Run updates
updateData()

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

No update Required


In [3]:
#get the fixture 
fixture, predictionDF = createFixture()

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "Round 3, 2025"
✔ Returning data for "Round 3, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "Round 2, 2025"
✔ Returning data for "Round 2, 2025" s]

shape: (1, 13)
┌─────┬─────┬─────┬────────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ id  ┆ pro ┆ nam ┆ abbrev ┆ nickn ┆ teamT ┆ club. ┆ club. ┆ club. ┆ club. ┆ club. ┆ round ┆ ByeFl │
│ --- ┆ vid ┆ e   ┆ iation ┆ ame   ┆ ype   ┆ id    ┆ provi ┆ name  ┆ abbre ┆ nickn ┆ .curr ┆ ag    │
│ i64 ┆ erI ┆ --- ┆ ---    ┆ ---   ┆ ---   ┆ ---   ┆ derId ┆ ---   ┆ viati ┆ ame   ┆ entRo ┆ ---   │
│     ┆ d   ┆ str ┆ str    ┆ str   ┆ str   ┆ i64   ┆ ---   ┆ str   ┆ on    ┆ ---   ┆ und   ┆ i32   │
│     ┆ --- ┆     ┆        ┆       ┆       ┆       ┆ str   ┆       ┆ ---   ┆ str   ┆ ---   ┆       │
│     ┆ str ┆     ┆        ┆       ┆       ┆       ┆   

In [4]:
#if fixture exists i.e no issues have been introduced, go ahead and use our models to get our predictions
modelWin = pickle.load(open('Data/winner.sav', 'rb'))

predictWinner = modelWin.predict(fixture).tolist()
probsWinner = modelWin.predict_proba(fixture).tolist()

#create the winner portion of the prediction df
predictionDF = (predictionDF
                 #place in results
                 .with_columns(pl.Series(predictWinner).alias('Predictions'),
                               pl.Series(probsWinner).alias('Probs'))
                 #Winner probs
                 .with_columns(((pl.col('Probs').list.get(pl.col('Predictions')))*100).round(2).alias('Win Prob'))
                 .with_columns(pl.when(pl.col('Predictions') == 1).then(pl.col('home.team.name')).otherwise(pl.col('away.team.name')).alias('Predicted Winner'))
                 #Add current round
                 .with_columns(pl.lit(getCurrentRoundNumbers()).alias('Round Number'))
                 #rename
                 .rename({'home.team.name' : 'Home Team',
                          'away.team.name' : 'Away Team'})
                 #keep only wants required
                 .select('Round Number', 'Home Team', 'Away Team', 'Predicted Winner', 'Win Prob')
                )


ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]



In [5]:
#Get the margin
#load the model in
model = pickle.load(open('Data/margin.sav', 'rb'))

#And the win probablities to fixture
fixture[['homeProb', 'awayProb']] = probsWinner
#predict the fixture
predictMargin = model.predict(fixture).tolist()

#Add to prediction df
predictionDF = (predictionDF.with_columns(pl.Series(predictMargin).round(0).cast(pl.Int32).alias('Margin')))


In [6]:
predictionDF

Round Number,Home Team,Away Team,Predicted Winner,Win Prob,Margin
i32,str,str,str,f64,i32
3,"""Essendon""","""Port Adelaide""","""Essendon""",63.21,64
3,"""Carlton""","""Western Bulldogs""","""Carlton""",56.9,3
3,"""Melbourne""","""Gold Coast Suns""","""Melbourne""",80.05,25
3,"""St Kilda""","""Richmond""","""St Kilda""",62.38,28
3,"""Hawthorn""","""GWS Giants""","""GWS Giants""",51.87,23
3,"""Brisbane Lions""","""Geelong Cats""","""Brisbane Lions""",61.54,28
3,"""Adelaide Crows""","""North Melbourne""","""Adelaide Crows""",68.78,28
3,"""West Coast Eagles""","""Fremantle""","""Fremantle""",60.96,48


In [7]:
#Write to folder
#Individual round tips
predictionDF.write_csv(fr'Tips/2025/Round_{getCurrentRoundNumbers()}.csv')

#Append to all_tips
with open("Tips/2025/All_Tips.csv", mode="a") as f:
   predictionDF.write_csv(f, include_header=False)

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

